In [2]:
path = 'C:/Users/joaqu/Box/Emotional-State-Capstone/data/R00/fitbit/fitbit_daily_summary/'
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt
import re
import os
import pickle
import datetime
import pendulum

In [126]:
files = os.listdir(path = path)
print(files)

#I have csv and xls files. Let's work first on the csv ones

['fitbit_export_R00_1002.csv', 'fitbit_export_R00_1004.xls', 'fitbit_export_R00_1008.csv', 'fitbit_export_R00_1009.csv', 'fitbit_export_R00_1014.csv', 'fitbit_export_R00_1016.csv', 'fitbit_export_R00_1018.csv', 'fitbit_export_R00_1019.csv', 'fitbit_export_R00_1020.csv', 'fitbit_export_R00_1021.csv', 'fitbit_export_R00_1024.csv', 'fitbit_export_R00_1029.csv', 'fitbit_export_R00_1030.csv', 'fitbit_export_R00_1032.csv', 'fitbit_export_R00_1034.csv', 'fitbit_export_R00_1035.csv', 'fitbit_export_R00_1037.csv', 'fitbit_export_R00_1039.csv', 'fitbit_export_R00_1041.csv', 'fitbit_export_R00_1045.csv', 'fitbit_export_R00_1047.csv', 'fitbit_export_R00_1048.csv', 'fitbit_export_R00_1049.csv', 'fitbit_export_R00_1050.csv', 'fitbit_export_R00_1051.csv', 'fitbit_export_R00_1055.csv', 'fitbit_export_R00_1060.csv', 'fitbit_export_R00_1066.csv', 'fitbit_export_R00_1067.csv', 'fitbit_export_R00_1073.csv', 'fitbit_export_R00_1079.csv', 'fitbit_export_R00_1080.csv', 'fitbit_export_R00_1081.csv', 'fitbit_e

In [94]:
def data_to_string(date):
    '''
    Function to transform the Date into the DATE format. This format is used in several tables. 
    We reccommend to use always the timestamp format for dates.
    '''
    if len(str(date.month)) == 1 :
        month =  '0' + str(date.month)
    else:
        month = str(date.month) 
    if len(str(date.day)) == 1: 
        day = '0' + str(date.day) 
    else:
        day = str(date.day)   
    return month+day

In [191]:
#ACTIVITY
activity = pd.read_csv(path + files[0], skiprows = 1, nrows=10)
activity['Date'] = pd.to_datetime(activity['Date'])
activity['DATE'] = activity['Date'].apply(data_to_string)
activity['ID'] = files[0][-8:-4] #lets use the ID in the file name
activity

,Date,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories,DATE,ID
0,2015-07-16,"4,200","35,439",15.56,53,280,327,166,162,"3,305",0716,1002
1,2015-07-17,"3,789","28,843",12.67,48,347,273,168,127,"2,794",0717,1002
2,2015-07-18,"4,478","39,139",17.24,102,340,400,124,194,"3,629",0718,1002
3,2015-07-19,"4,715","45,343",19.94,116,327,391,121,248,"3,920",0719,1002
4,2015-07-20,"3,948","31,182",13.72,61,407,335,114,157,"3,001",0720,1002
5,2015-07-21,"3,773","35,138",15.53,141,445,201,60,233,"2,705",0721,1002
6,2015-07-22,"3,798","31,882",14.02,61,391,349,99,146,"2,842",0722,1002
7,2015-07-23,"4,023","35,056",15.41,79,344,320,119,176,"3,093",0723,1002
8,2015-07-24,"3,862","31,462",13.87,67,434,273,126,167,"2,876",0724,1002
9,2015-07-25,"4,684","45,983",20.37,119,381,445,101,226,"3,901",0725,1002


In [192]:
ACTIVITY_R00 = activity.copy() #initialize acivity df

In [193]:
#SLEEP
sleep = pd.read_csv(path + files[0], skiprows = 14)
sleep = sleep.drop(columns = [i for i in sleep.columns if i[:7] == 'Unnamed']).copy()
sleep['Date'] = pd.to_datetime(sleep['Date'])
to_DATE = sleep['Date'].apply(data_to_string)
sleep['DATE'] = to_DATE
sleep['ID'] = files[0][-8:-4] #lets use the ID in the file name
SLEEP_R00 = sleep.copy()

In [194]:
SLEEP_R00 #initialize sleep df

,Date,Minutes Asleep,Minutes Awake,Number of Awakenings,Time in Bed,DATE,ID
0,2015-07-16,477,20,16,505,0716,1002
1,2015-07-17,404,45,17,462,0717,1002
2,2015-07-18,361,17,13,382,0718,1002
3,2015-07-19,281,9,4,291,0719,1002
4,2015-07-20,409,18,15,427,0720,1002
5,2015-07-21,474,24,17,501,0721,1002
6,2015-07-22,435,19,10,455,0722,1002
7,2015-07-23,461,20,17,481,0723,1002
8,2015-07-24,413,24,13,440,0724,1002
9,2015-07-25,262,22,15,287,0725,1002


In [163]:
#It seems that is working. Let's try it with all the subjects

In [164]:
csv_files = [i for i in files if i[-3:] == 'csv']
for n, file in enumerate(csv_files[1:]): #I already did the first file
    
    activity = pd.read_csv(path + file, skiprows = 1, nrows=10)
    activity['Date'] = pd.to_datetime(activity['Date'])
    activity['DATE'] = activity['Date'].apply(data_to_string)
    activity['ID'] = file[-8:-4]

    
    sleep = pd.read_csv(path + file, skiprows = 14)
    sleep = sleep.drop(columns = [i for i in sleep.columns if i[:7] == 'Unnamed']).copy()
    sleep['Date'] = pd.to_datetime(sleep['Date'])
    to_DATE = sleep['Date'].apply(data_to_string)
    sleep['DATE'] = to_DATE
    sleep['ID'] = file[-8:-4]    


KeyError: 'Date'

In [158]:
#I have an error on the file:
file

'fitbit_export_R00_1084.csv'

In [ ]:
#The file didn't have the same number of rows than the others. I modified this manually.

The same happend for the files:<br>
- fitbit_export_R00_1081.csv<br>
- fitbit_export_R00_1084.csv<br>
- fitbit_export_R00_1087.csv<br>

These files were modified by hand

In [165]:
file

'fitbit_export_R00_1084.csv'

has more measurements than the other subjects (12 instead of 14). I will add an exception for this file

In [195]:
for n, file in enumerate(csv_files[1:]): #I already did the first file
    if file[-8:-4] == '1084': 
        skip = 16
        nrows = 12
    else: 
        skip = 14
        nrows = 10
    
    activity = pd.read_csv(path + file, skiprows = 1, nrows=nrows)
    activity['Date'] = pd.to_datetime(activity['Date'])
    activity['DATE'] = activity['Date'].apply(data_to_string)
    activity['ID'] = file[-8:-4]
    ACTIVITY_R00 =  ACTIVITY_R00.append(activity, ignore_index=True)

    sleep = pd.read_csv(path + file, skiprows = skip)
    sleep = sleep.drop(columns = [i for i in sleep.columns if i[:7] == 'Unnamed']).copy()
    sleep['Date'] = pd.to_datetime(sleep['Date'])
    to_DATE = sleep['Date'].apply(data_to_string)
    sleep['DATE'] = to_DATE
    sleep['ID'] = file[-8:-4]
    SLEEP_R00 = SLEEP_R00.append(sleep, ignore_index=True)

In [196]:
ACTIVITY_R00

,Date,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories,DATE,ID
0,2015-07-16,"4,200","35,439",15.56,53,280,327,166,162,"3,305",0716,1002
1,2015-07-17,"3,789","28,843",12.67,48,347,273,168,127,"2,794",0717,1002
2,2015-07-18,"4,478","39,139",17.24,102,340,400,124,194,"3,629",0718,1002
3,2015-07-19,"4,715","45,343",19.94,116,327,391,121,248,"3,920",0719,1002
4,2015-07-20,"3,948","31,182",13.72,61,407,335,114,157,"3,001",0720,1002
...,...,...,...,...,...,...,...,...,...,...,...,...
367,2016-11-22,"2,358","6,430",2.80,15,589,257,0,0,959,1122,1097
368,2016-11-23,"1,984","3,208",1.40,14,"1,040",125,0,0,461,1123,1097
369,2016-11-24,"2,123","3,630",1.58,12,739,195,0,0,662,1124,1097
370,2016-11-25,"2,333","8,928",3.89,19,766,252,0,0,963,1125,1097


Now we need to add the xls files

In [199]:
xls_files = [i for i in files if i[-3:] == 'xls']
xls_files

['fitbit_export_R00_1004.xls']

In [209]:
file = pd.read_excel(path + xls_files[0],sheet_name=None)
activity = file['Activities']
activity['Date'] = pd.to_datetime(activity['Date'])
activity['DATE'] = activity['Date'].apply(data_to_string)
activity['ID'] = xls_files[0][-8:-4]
ACTIVITY_R00 =  ACTIVITY_R00.append(activity, ignore_index=True)

In [211]:
activity.head() #checking everything is ok

,Date,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories,DATE,ID
0,2015-06-30,"4,458","18,697",11.86,13,451,350,32,91,"3,059",0630,1004
1,2015-07-01,"3,188","11,016",6.87,7,692,224,5,33,"1,507",0701,1004
2,2015-07-02,"3,502","11,677",5.53,2,711,337,28,22,"1,988",0702,1004
3,2015-07-03,"3,780","6,166",2.92,5,715,302,61,36,"2,194",0703,1004
4,2015-07-04,"3,435","9,021",4.27,7,694,344,21,15,"1,854",0704,1004


In [213]:
sleep = file['Sleep']
sleep['Date'] = pd.to_datetime(sleep['Date'])
to_DATE = sleep['Date'].apply(data_to_string)
sleep['DATE'] = to_DATE
sleep['ID'] = xls_files[0][-8:-4]
SLEEP_R00 = SLEEP_R00.append(sleep, ignore_index=True)

In [214]:
sleep

,Date,Minutes Asleep,Minutes Awake,Number of Awakenings,Time in Bed,DATE,ID
0,2015-06-30,479,37,21,516,0630,1004
1,2015-07-01,427,18,10,456,0701,1004
2,2015-07-02,258,25,15,287,0702,1004
3,2015-07-03,342,4,3,352,0703,1004
4,2015-07-04,340,20,11,366,0704,1004
5,2015-07-05,330,34,21,386,0705,1004
6,2015-07-06,460,17,13,479,0706,1004
7,2015-07-07,440,20,9,474,0707,1004
8,2015-07-08,74,7,4,86,0708,1004
9,2015-07-09,184,5,5,194,0709,1004


In [218]:
with open('./Clean Data R00/ACTIVITY_R00.pickle', 'wb') as handle:
    pickle.dump(ACTIVITY_R00, handle)

In [217]:
with open('./Clean Data R00/SLEEP_R00.pickle', 'wb') as handle:
    pickle.dump(SLEEP_R00, handle)

I realize that I did not added the experiment column, this is going to be necessary for the future.

In [3]:
with open ('./Clean Data R00/ACTIVITY_R00.pickle','rb') as activity:
    ACTIVITY_R00 = pickle.load(activity)

In [9]:
with open ('./Clean Data R00/SLEEP_R00.pickle','rb') as sleep:
    SLEEP_R00 = pickle.load(sleep)

In [10]:
ACTIVITY_R00['Experiment'] = 'R00'
SLEEP_R00['Experiment'] = 'R00'

In [12]:
SLEEP_R00.head()

,Date,Minutes Asleep,Minutes Awake,Number of Awakenings,Time in Bed,DATE,ID,Experiment
0,2015-07-16,477,20,16,505,0716,1002,R00
1,2015-07-17,404,45,17,462,0717,1002,R00
2,2015-07-18,361,17,13,382,0718,1002,R00
3,2015-07-19,281,9,4,291,0719,1002,R00
4,2015-07-20,409,18,15,427,0720,1002,R00


In [13]:
ACTIVITY_R00.head()

,Date,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories,DATE,ID,Experiment
0,2015-07-16,"4,200","35,439",15.56,53,280,327,166,162,"3,305",0716,1002,R00
1,2015-07-17,"3,789","28,843",12.67,48,347,273,168,127,"2,794",0717,1002,R00
2,2015-07-18,"4,478","39,139",17.24,102,340,400,124,194,"3,629",0718,1002,R00
3,2015-07-19,"4,715","45,343",19.94,116,327,391,121,248,"3,920",0719,1002,R00
4,2015-07-20,"3,948","31,182",13.72,61,407,335,114,157,"3,001",0720,1002,R00


In [14]:
with open('./Clean Data R00/ACTIVITY_R00.pickle', 'wb') as handle:
    pickle.dump(ACTIVITY_R00, handle)
with open('./Clean Data R00/SLEEP_R00.pickle', 'wb') as handle:
    pickle.dump(SLEEP_R00, handle)